In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
import holodeck.detstats as det
from holodeck.constants import YR, MSOL
from holodeck import utils


import hasasia.sensitivity as hsen
import hasasia.sim as hsim

## Goal:
Pass in an hdf5 file from a library of param spaces and return the average detection probability.

## Steps:
1) Read in file: fobs, hc_ss, hc_bg
2) Produce make a PTA set-up calibrated to 50% hc_bg DP?
3) Given pulsar setup and hdf file, calculate average ss DP and average bg DP
    * Calculate bg DP for each of R strain realizations
    * Build S sky realizations
    * Calculate ss DP for each of S sky realizations for each of R strain realizations.


In [59]:
ssfile = h5py.File('/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss12_n10_r10_f20_l5_p1_s10/ss_lib.hdf5', 'r')
print(list(ssfile.keys()))
# print(ssfile['pta_dur'][0])
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
dur = ssfile['pta_dur'][0]
cad = ssfile['pta_cad'][0]
hc_ss_all = ssfile['hc_ss'][...]
hc_bg_all = ssfile['hc_bg'][...]
ssfile.close()

['bgpar', 'dfobs', 'fobs', 'hc_bg', 'hc_ss', 'pta_cad', 'pta_dur', 'sample_params', 'sspar']


## single realization example

### build PTA
This is just used to store theta, phi, and sigmas of the pta.

[9.88399495e-10 9.88399495e-10 9.88399495e-10 9.88399495e-10
 9.88399495e-10 9.88399495e-10 9.88399495e-10 9.88399495e-10
 9.88399495e-10 9.88399495e-10 9.88399495e-10 9.88399495e-10
 9.88399495e-10 9.88399495e-10 9.88399495e-10 9.88399495e-10
 9.88399495e-10 9.88399495e-10 9.88399495e-10 9.88399495e-10]


## Step 3: Given pulsar setup, and hdf file, calculate average ss DP

Parameters and functions needed:
* def detect_bg_pta(
    - pulsars, 
    - spectra, 
    - cad, 
    - hc_bg, 
    - alpha_0=0.001, 
    - ret_all = False):
* def detect_ss_pta(
    - pulsars, 
    - cad, 
    - dur, 
    - fobs,
    - dfobs, 
    - hc_ss, 
    - hc_bg
    - theta_ss=None, 
    - phi_ss=None, 
    - Phi0_ss=None, 
    - iota_ss=None, 
    - psi_ss=None):